<a href="https://colab.research.google.com/github/sruthi-v04/docu3c_submission/blob/main/Bollywood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
import pandas as pd

In [181]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **TEXT CLASSIFICATION**

**Extraction of text from pdf**

In [182]:
!pip install PyPDF2
!pip install langid

In [183]:
import os
import PyPDF2
import langid
import pandas as pd

In [184]:
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ''
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

In [185]:
def detect_language(text):
    return langid.classify(text)

In [186]:
import re
import nltk
nltk.download('words')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import words

def remove_html(text):
  html_tag= re.compile('<.*?>')
  text= re.sub(html_tag, '', text)
  special_char= re.compile('[^a-zA-z\s]')
  text= re.sub(special_char, '', text)
  new_line= re.compile('[\nNtT]')
  text= re.sub(new_line, '', text)
  valid_words= set(words.words())
  word_in_text= word_tokenize(text)
  filtered= [token for token in word_in_text if token.lower() in valid_words]
  text= ' '.join(filtered)


  return text


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [187]:
pdf_directory = '/content/drive/MyDrive/bollywood_data/Bollywood-Data-master/scripts-data'

# DataFrame to store results
pdf_data = pd.DataFrame(columns=['PDF_Name', 'Extracted_Text'])

# Iterate through PDF files
for pdf_file in os.listdir(pdf_directory):
    if pdf_file.endswith('.pdf'):
        # Extract text from PDF
        pdf_text = extract_text_from_pdf(os.path.join(pdf_directory, pdf_file))
        english_text= remove_html(pdf_text)


        # Add data to DataFrame
        pdf_data = pdf_data.append({'PDF_Name': pdf_file.lower(), 'Extracted_Text': english_text}, ignore_index=True)

<ipython-input-187-24760c8f5740>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_data = pdf_data.append({'PDF_Name': pdf_file.lower(), 'Extracted_Text': english_text}, ignore_index=True)
<ipython-input-187-24760c8f5740>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_data = pdf_data.append({'PDF_Name': pdf_file.lower(), 'Extracted_Text': english_text}, ignore_index=True)
<ipython-input-187-24760c8f5740>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  pdf_data = pdf_data.append({'PDF_Name': pdf_file.lower(), 'Extracted_Text': english_text}, ignore_index=True)
<ipython-input-187-24760c8f5740>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

**NLP on the extracted text**

In [188]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [189]:
lemmatizer= WordNetLemmatizer()

In [190]:
nltk.download('stopwords')
from nltk.corpus import stopwords
sw= stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [191]:
def remove_sw(tex):
  text_split= tex.split()
  text_split_lower = [word.lower() for word in text_split]
  lemmatized= [lemmatizer.lemmatize(rem) for rem in text_split_lower]
  sw_removed= [word for word in lemmatized if word not in sw]
  text= ' '.join(sw_removed)
  return text


In [192]:
pdf_data['new']= pdf_data['Extracted_Text'].apply(remove_sw)

In [193]:
pdf_data['new'].head(1)

0    fade mids insurgency peak defence challenge formed play key role reducing valley decided hold six show world ha fade day lonely road wo couple ak peep barbed bunker building posh area super bureau road conference room brigadier handful senior army police brigadier bulbul insurgency force bulbul arm fund run force able full brigadier k individual k f individual group anyone proxy war hey primary senior cop general police pronounce b brigadier army run bu crucial brigadier see urn smug smile brigadier bulbul sar singing kill ex chamber day man dressed wearing ha window handsome lawyer old man behind speak k wa u j u wa one gave weapon hide guy informed army armed special god heaven army army armed special acold j u son cinema help please ge uncle free boy ge bu possibly save life move camp ge prison well need formal need ge deep ill false would ge hearing disappear man closer k wha shall public arm possession old man confused ex gypsy wo moving wo one arm possession envelope aside b

**create a misogyny classification model**

In [194]:
miso=[0,1,0,0,1,0,0,0,1,0,1,0,0]

In [195]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Assuming 'new' is the lemmatized text column in your DataFrame pdf_data
lemmatized_text_column = pdf_data['new']

# Create an empty DataFrame to store TF-IDF vectors
tfidf_vectors_df = pd.DataFrame()

# Create a TF-IDF vectorizer
tfid_vect = TfidfVectorizer(max_features=500, stop_words=None)

# Iterate through each row and create TF-IDF vector
for index, lemmatized_text in lemmatized_text_column.iteritems():
    tf_features = tfid_vect.fit_transform([lemmatized_text])
    tfidf_vector = pd.DataFrame(tf_features.toarray(), columns=tfid_vect.get_feature_names_out())
    tfidf_vector.index = [index]  # Set the index to match the original DataFrame
    tfidf_vectors_df = pd.concat([tfidf_vectors_df, tfidf_vector])

# Add the 'vectors' column to the original DataFrame
pdf_data['vectors'] = tfidf_vectors_df.values.tolist()

<ipython-input-195-19d74a9f9df6>:14: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, lemmatized_text in lemmatized_text_column.iteritems():


In [196]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import numpy as np

# Assuming 'classification_column' is the label column
X = np.vstack(pdf_data['vectors'])
y = miso

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Multinomial Naive Bayes classifier
classifier = MultinomialNB()

# Train the classifier
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 1.0
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         3

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3

Confusion Matrix:
[[3]]


In [197]:
pd.set_option('display.max_colwidth', 50)

**Usage of average relation dataset**

In [198]:
avg_relation= pd.read_csv('/content/drive/MyDrive/bollywood_data/Bollywood-Data-master/wikipedia-data/avg_relation - avg_wv_relation.csv')

In [199]:
avg_relation.head(30)

,character_movie,gender,features,vector
0,woman_Aan Milo Sajna,FEMALE,young,-0.105138 -0.029248 -0.063298 -0.007706 -0.180...
1,Anil_Aan Milo Sajna,MALE,NaN,0
2,Savitri_Aan Milo Sajna,FEMALE,NaN,0
3,Deepali_Aan Milo Sajna,FEMALE,NaN,0
4,Shekar_Abhinetri,UNKNOWN,NaN,0
5,Anjana_Abhinetri,FEMALE,NaN,0
6,Nekichand_Bachpan,unknown,NaN,0
7,father_Bachpan,MALE,unhappilly,0
8,Rahim_Bachpan,MALE,"father, father, father",-2.82591750e-01 6.52440000e-02 -2.36074500...
9,Kashi_Bachpan,UNKNOWN,innocent,-1.03062000e-01 1.66135000e-01 -2.67929000...


**Sentiment of Women characters dialogues**

In [200]:
avg_relation[['Character', 'Movie']] = avg_relation['character_movie'].str.split('_', expand=True)


In [201]:
avg_relation.head()

,character_movie,gender,features,vector,Character,Movie
0,woman_Aan Milo Sajna,FEMALE,young,-0.105138 -0.029248 -0.063298 -0.007706 -0.180...,woman,Aan Milo Sajna
1,Anil_Aan Milo Sajna,MALE,NaN,0,Anil,Aan Milo Sajna
2,Savitri_Aan Milo Sajna,FEMALE,NaN,0,Savitri,Aan Milo Sajna
3,Deepali_Aan Milo Sajna,FEMALE,NaN,0,Deepali,Aan Milo Sajna
4,Shekar_Abhinetri,UNKNOWN,NaN,0,Shekar,Abhinetri


In [202]:
female_df = avg_relation[avg_relation['gender'] == 'FEMALE']

In [203]:
female_df.head()

,character_movie,gender,features,vector,Character,Movie
0,woman_Aan Milo Sajna,FEMALE,young,-0.105138 -0.029248 -0.063298 -0.007706 -0.180...,woman,Aan Milo Sajna
2,Savitri_Aan Milo Sajna,FEMALE,NaN,0,Savitri,Aan Milo Sajna
3,Deepali_Aan Milo Sajna,FEMALE,NaN,0,Deepali,Aan Milo Sajna
5,Anjana_Abhinetri,FEMALE,NaN,0,Anjana,Abhinetri
13,Seema_Chetna,FEMALE,unrecognizable,0,Seema,Chetna


In [204]:
import numpy as np
import pandas as pd



female_df['vector'] = female_df['vector'].apply(lambda x: np.array([float(val) for val in x.split()]))


max_length = female_df['vector'].apply(len).max()

female_df['vector'] = female_df['vector'].apply(lambda x: np.pad(x, (0, max_length - len(x))))

female_df['vector'] = female_df['vector'].apply(lambda x: x.reshape(1, -1))

average_vectors = female_df.groupby('Movie')['vector'].apply(lambda x: np.mean(np.vstack(x), axis=0)).reset_index()

<ipython-input-204-fda72070178b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  female_df['vector'] = female_df['vector'].apply(lambda x: np.array([float(val) for val in x.split()]))
<ipython-input-204-fda72070178b>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  female_df['vector'] = female_df['vector'].apply(lambda x: np.pad(x, (0, max_length - len(x))))
<ipython-input-204-fda72070178b>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

In [205]:
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd


vectors = np.vstack(female_df['vector'])
vectors = vectors.astype(float)


num_clusters = 2


kmeans = KMeans(n_clusters=num_clusters, random_state=42)
female_df['cluster'] = kmeans.fit_predict(vectors)


cluster_centers = kmeans.cluster_centers_

positive_cluster = np.argmax(np.mean(cluster_centers, axis=1))
negative_cluster = 1 - positive_cluster


female_df['sentiment'] = np.where(female_df['cluster'] == positive_cluster, 'Positive', 'Negative')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-205-bf95355d7ce2>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  female_df['cluster'] = kmeans.fit_predict(vectors)
<ipython-input-205-bf95355d7ce2>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  female_df['sentiment'] = np.where(female_df['cluster'] == posi

In [206]:
pd.set_option('display.max_colwidth', 2000)

In [207]:
vector_length = len(female_df['vector'].iloc[1])
print(vector_length)

1


In [208]:
female_df.head()

,character_movie,gender,features,vector,Character,Movie,cluster,sentiment
0,woman_Aan Milo Sajna,FEMALE,young,"[[-0.105138, -0.029248, -0.063298, -0.007706, -0.180894, -0.104018, 0.034577, -0.051221, -0.129727, -0.031833, -0.178217, 0.19768, -0.0573, -0.059109, -0.041048, -0.043887, 0.135037, 0.300367, 0.145015, 0.031881, -0.205729, -0.037333, 0.307861, -0.114496, 0.124391, 0.099326, -0.091438, 0.128143, 0.134054, 0.006646, 0.060022, -0.083128, -0.107062, -0.038827, 0.135397, 0.119597, 0.119368, 0.00618, 0.004359, 0.009318, 0.12306, 0.160797, -0.14984, 0.041717, 0.165998, 0.02331, -0.006929, 0.242037, -0.19263, -0.130126, -0.081618, -0.208262, -0.087869, -0.053085, 0.030829, -0.010559, -0.045659, -0.137754, 0.092101, -0.207898, 0.244597, -0.043084, 0.121871, -0.080083, -0.083782, -0.067626, 0.039911, 0.058039, 0.094585, 0.138638, 0.057615, 0.082765, 0.018154, 0.11145, 0.048383, 0.200493, -0.179834, -0.088518, -0.130542, -0.043079, -0.022001, 0.081154, -0.11968, -0.058292, 0.142399, -0.144937, 0.035644, -0.249231, 0.082367, -0.168491, 0.026903, 0.002506, -0.299859, -0.025856, -0.242518, -0.312798, -0.1355, 0.018445, -0.162201, -0.12209, ...]]",woman,Aan Milo Sajna,0,Negative
2,Savitri_Aan Milo Sajna,FEMALE,NaN,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]]",Savitri,Aan Milo Sajna,1,Positive
3,Deepali_Aan Milo Sajna,FEMALE,NaN,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]]",Deepali,Aan Milo Sajna,1,Positive
5,Anjana_Abhinetri,FEMALE,NaN,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]]",Anjana,Abhinetri,1,Positive
13,Seema_Chetna,FEMALE,unrecognizable,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...]]",Seema,Chetna,1,Positive


In [209]:
sentiment_df = female_df[female_df['vector'].apply(lambda x: not np.allclose(x, 0.0))]





In [210]:
sentiment_df.head(20)

,character_movie,gender,features,vector,Character,Movie,cluster,sentiment
0,woman_Aan Milo Sajna,FEMALE,young,"[[-0.105138, -0.029248, -0.063298, -0.007706, -0.180894, -0.104018, 0.034577, -0.051221, -0.129727, -0.031833, -0.178217, 0.19768, -0.0573, -0.059109, -0.041048, -0.043887, 0.135037, 0.300367, 0.145015, 0.031881, -0.205729, -0.037333, 0.307861, -0.114496, 0.124391, 0.099326, -0.091438, 0.128143, 0.134054, 0.006646, 0.060022, -0.083128, -0.107062, -0.038827, 0.135397, 0.119597, 0.119368, 0.00618, 0.004359, 0.009318, 0.12306, 0.160797, -0.14984, 0.041717, 0.165998, 0.02331, -0.006929, 0.242037, -0.19263, -0.130126, -0.081618, -0.208262, -0.087869, -0.053085, 0.030829, -0.010559, -0.045659, -0.137754, 0.092101, -0.207898, 0.244597, -0.043084, 0.121871, -0.080083, -0.083782, -0.067626, 0.039911, 0.058039, 0.094585, 0.138638, 0.057615, 0.082765, 0.018154, 0.11145, 0.048383, 0.200493, -0.179834, -0.088518, -0.130542, -0.043079, -0.022001, 0.081154, -0.11968, -0.058292, 0.142399, -0.144937, 0.035644, -0.249231, 0.082367, -0.168491, 0.026903, 0.002506, -0.299859, -0.025856, -0.242518, -0.312798, -0.1355, 0.018445, -0.162201, -0.12209, ...]]",woman,Aan Milo Sajna,0,Negative
28,Janki_Geet,FEMALE,"sister, sister, sister","[[-0.280121, -0.002382, -0.197149, -0.063887, -0.510019, -0.248295, 0.127471, -0.120913, -0.332435, -0.0564, -0.523231, 0.545695, -0.140668, -0.183122, -0.022088, -0.047358, 0.342626, 0.828918, 0.25496, -0.010146, -0.672146, -0.079536, 0.697625, -0.278186, 0.120319, 0.079212, -0.210055, 0.232158, -0.022389, -0.100141, 0.144298, -0.02803, -0.271797, -0.065738, 0.109224, 0.265732, 0.381423, -0.192735, 0.084359, 0.133534, 0.051327, -0.026448, 0.078249, 0.082275, 0.232291, -0.176336, -0.077679, 0.093303, -0.078861, -0.187826, 0.078296, -0.287562, 0.13407, -0.092215, 0.003478, 0.066426, -0.104592, -0.030455, -0.105492, -0.414195, 0.2562, -0.023927, -0.040942, -0.147882, -0.142632, -0.125066, -0.160486, 0.176114, 0.251171, -0.039142, -0.207373, -0.134918, 0.081226, 0.293861, 0.032205, 0.27096, 0.008982, -0.170641, -0.092746, 0.206816, 0.116979, 0.065593, -0.293989, -0.037, 0.250089, -0.143829, 0.105228, -0.501883, -0.087251, 0.269726, 0.025594, 0.102548, -0.547274, 0.002462, -0.323322, -0.41374, -0.443747, -0.010987, -0.206775, -0.103604, ...]]",Janki,Geet,0,Negative
53,Deepa_Jeevan Mrityu,FEMALE,"mother, mother, mother","[[-0.296385, -0.05633325, -0.13522125, -0.00415425, -0.59865, -0.30668175, 0.176706, -0.15085875, -0.3269295, -0.04254375, -0.550512, 0.58002975, -0.137118, -0.2378025, -0.0273675, -0.059994, 0.404844, 0.8796765, 0.23359875, -0.01250475, -0.77484675, -0.10154475, 0.70202925, -0.223812, 0.13391325, 0.087546, -0.20137275, 0.22713675, -0.120366, -0.12440175, 0.143619, -0.0266115, -0.257919, -0.069366, 0.09985875, 0.28501875, 0.369966, -0.2362815, 0.09668475, 0.13703325, 0.01310475, -0.06455775, 0.1438125, 0.0743805, 0.2052765, -0.212337, -0.063468, 0.04036725, -0.02630925, -0.1516245, 0.100137, -0.249726, 0.176361, -0.07509225, 0.00208125, 0.0739455, -0.12322425, -0.0149115, -0.1469295, -0.374097, 0.21338475, -0.010689, -0.07994625, -0.142386, -0.1137585, -0.11118675, -0.19072125, 0.16368, 0.24465225, -0.0717315, -0.23284125, -0.17176275, 0.10854675, 0.33829425, 0.0333915, 0.2117265, 0.07445325, -0.160539, -0.09618075, 0.22426575, 0.11581875, 0.045924, -0.28256925, -0.03069075, 0.2146965, -0.09776775, 0.0879915, -0.45274125, -0.13851525, 0.369876, 0.00074325, 0.1016355, -0.49835775, 0.018651, -0.285366, -0.35355375, -0.44370375, -0.03338175, -0.20054175, -0.10286325, ...]]",Deepa,Jeevan Mrityu,0,Negative
58,Poonam_Kati Patang,FEMALE,real,"[[-0.072086, 0.1972, -0.270346, -0.19603, 0.060851, 0.073271, -0.107821, 0.048724, -0.158331, -0.065398, -0.16424, 0.134049, -0.078994, 0.090732, -0.014168, 0.049108, -0.05846, 0.168687, 0.19259, -0.008668, 0.000704, 0.017404, 0.324382, -0.295161, 0.028492, -0.034267, -0.12992, 0.087761, 0.171675, 0.040038, 0.078427, -0.008976, -0.149176, -0.007805, 0.083726, 0.057529

**Finding the Cosine similarity betwween the male and the female sentences**

In [211]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [212]:
malecos_df = avg_relation[avg_relation['gender'] == 'MALE']
femalecos_df= avg_relation[avg_relation['gender']== 'FEMALE']

In [213]:
malecos_df.head()

,character_movie,gender,features,vector,Character,Movie
1,Anil_Aan Milo Sajna,MALE,NaN,0,Anil,Aan Milo Sajna
7,father_Bachpan,MALE,unhappilly,0,father,Bachpan
8,Rahim_Bachpan,MALE,"father, father, father",-2.82591750e-01 6.52440000e-02 -2.36074500e-01 -1.02521250e-01 -4.85349000e-01 -2.14914000e-01 1.14368250e-01 -1.00077000e-01 -3.39276750e-01 -5.80237500e-02 -5.46981000e-01 5.43957000e-01 -1.47099750e-01 -1.62153750e-01 -7.88175000e-03 -2.25352500e-02 3.04674750e-01 8.14780500e-01 2.63843250e-01 -3.38235000e-02 -6.62127000e-01 -6.27510000e-02 7.18195500e-01 -3.07855500e-01 9.68355000e-02 3.05370000e-02 -2.25795750e-01 2.20697250e-01 -1.19085750e-01 -1.15635750e-01 1.57271250e-01 6.87600000e-03 -2.80319250e-01 -5.86177500e-02 7.16850000e-02 2.65853250e-01 4.17924000e-01 -2.65039500e-01 1.00776000e-01 1.59833250e-01 3.23790000e-02 -5.93752500e-02 1.24941750e-01 9.00720000e-02 2.28087000e-01 -2.21615250e-01 -9.03157500e-02 5.16510000e-02 -4.60515000e-02 -1.94586000e-01 1.15743750e-01 -2.82369750e-01 1.81963500e-01 -8.27265000e-02 1.01700000e-02 7.43527500e-02 -9.99772500e-02 1.16250000e-02 -1.37325000e-01 -4.27012500e-01 2.37943500e-01 -1.47420000e-02 -8.65582500e-02 -1.70416500e-01 -1.43304750e-01 -1.37024250e-01 -1.99973250e-01 1.80834750e-01 2.68645500e-01 -1.07578500e-01 -2.66682750e-01 -1.81590000e-01 9.25110000e-02 2.83446750e-01 2.42797500e-02 2.71885500e-01 6.02445000e-02 -1.92814500e-01 -4.46595000e-02 2.65848000e-01 1.41623250e-01 6.26422500e-02 -3.12324750e-01 -3.36060000e-02 2.68870500e-01 -1.36284750e-01 1.03037250e-01 -5.13213750e-01 -1.00074000e-01 2.94009750e-01 2.48910000e-02 1.23155250e-01 -5.41509000e-01 8.83500000e-03 -3.03912000e-01 -3.95143500e-01 -4.72934250e-01 -2.84587500e-02 -2.09351250e-01 -7.12882500e-02 1.22754000e-01 4.19838750e-01 3.94311000e-01 -2.63352000e-01 3.89088000e-01 1.20441000e-01 4.19309250e-01 3.16717500e-01 4.16472000e-01 4.44000000e-01 2.30760000e-01 -4.92592500e-02 5.95315500e-01 -3.71184000e-01 -1.49985000e-02 2.36963250e-01 4.37353500e-01 5.633...,Rahim,Bachpan
10,Tom_Bachpan,MALE,NaN,0,Tom,Bachpan
12,actor_Bombay Talkie,MALE,famous,0.022116 -0.270536 0.235497 0.218431 -0.129693 -0.160498 0.071661 -0.089417 0.059936 0.016141 0.087965 -0.013129 0.042521 -0.111169 -0.09021 -0.099235 0.169962 0.008531 -0.001163 0.103093 -0.035753 -0.058877 -0.046496 0.192741 0.208268 0.236214 0.045453 0.066766 0.223007 0.057534 -0.025934 -0.181603 0.062618 -0.04117 0.197828 0.035166 -0.183418 0.207999 -0.059802 -0.116154 0.129994 0.288584 -0.292029 -0.02132 0.060363 0.195667 0.1044 0.316812 -0.243482 0.004717 -0.234326 -0.080662 -0.266731 -0.002066 0.030999 -0.067242 -0.04494 -0.25842 0.199613 0.069868 0.159815 -0.04006 0.23711 0.027474 0.013507 0.029423 0.196669 -0.0627 -0.07933 0.319347 0.32474 0.268122 -0.002369 0.102195 0.064005 0.003411 -0.28163 0.035198 -0.25065 -0.318558 -0.190688 0.058996 0.08647 -0.055714 -0.060214 -0.046384 -0.047532 0.07783 0.13827 -0.36398 -0.014301 -0.116968 -0.032382 -0.023535 -0.141688 -0.146802 0.14811 0.028485 -0.107189 -0.195908 -0.111216 -0.016611 -0.151599 0.01871 -0.03737 0.19281 -0.13015 0.012082 -0.035549 -0.120507 0.157299 0.031929 -0.093093 -0.201246 -0.133834 -0.031172 -0.033015 0.019326 0.068596 -0.058714 -0.263521 -0.035104 0.279522 0.020757 0.098447 -0.012484 -0.029198 0.000794 0.022144 0.035799 -0.075571 -0.129707 0.122654 0.154805 0.082939 0.128479 0.05608 -0.151387 -0.203828 -0.034486 0.21303 0.07586 -0.188525 -0.129057 0.066052 -0.009558 -0.075885 0.026886 -0.405358 -0.265887 0.028862 -0.043643 0.116387 0.082442 -0.247565 -0.28901 -0.135495 -0.045696 -0.244657 -0.019139 -0.099487 -0.096775 0.237141 0.084421 -0.018128 0.09209 -0.054095 0.128816 0.166408 -0.087505 0.06611 0.07302 0.178469 -0.227673 0.109447 0.079641 -0.008022 0.011572 0.260735 -0.159274 0.125698 0.042547 -0.030789 -0.147069 0.276993 0.137312 0.091731 -0.019391 0.365554 0.082727 0.165423 0.033059 -0.106933 0.231128 0.168066 -0.13508 0.225739 0.064803 -0.155822 -0.164407,actor,Bombay Talk

In [214]:
misogyny_results = pd.DataFrame(columns=['Movie', 'Cosine_Similarity'])

In [215]:
import numpy as np


misogyny_results = pd.DataFrame(columns=['Movie', 'Cosine_Similarity'])

misogyny_results = pd.DataFrame(columns=['Movie', 'Cosine_Similarity'])


default_similarity = -1.0


for movie_title in avg_relation['Movie'].unique():

    female_vectors_str = femalecos_df[(femalecos_df['Movie'] == movie_title) & ~femalecos_df['vector'].isna()]['vector'].tolist()
    male_vectors_str = malecos_df[(malecos_df['Movie'] == movie_title) & ~malecos_df['vector'].isna()]['vector'].tolist()

    if len(female_vectors_str) > 0 and len(male_vectors_str) > 0:

        female_vectors = [list(map(float, vector.split())) if isinstance(vector, str) else vector for vector in female_vectors_str]
        male_vectors = [list(map(float, vector.split())) if isinstance(vector, str) else vector for vector in male_vectors_str]


        if any(not np.allclose(np.zeros_like(vector), vector) for vector in female_vectors):

            max_length = max(len(vector) for vector in female_vectors + male_vectors)
            female_vectors = [vector + [0] * (max_length - len(vector)) for vector in female_vectors]
            avg_female_vector = np.mean(female_vectors, axis=0)
        else:
            avg_female_vector = None

        if any(not np.allclose(np.zeros_like(vector), vector) for vector in male_vectors):

            max_length = max(len(vector) for vector in female_vectors + male_vectors)
            male_vectors = [vector + [0] * (max_length - len(vector)) for vector in male_vectors]
            avg_male_vector = np.mean(male_vectors, axis=0)
        else:
            avg_male_vector = None


        if avg_female_vector is not None and avg_male_vector is not None:
            cos_similarity = cosine_similarity([avg_female_vector], [avg_male_vector])[0, 0]
        else:
            cos_similarity = default_similarity

        misogyny_results = misogyny_results.append({'Movie': movie_title, 'Cosine_Similarity': cos_similarity}, ignore_index=True)

<ipython-input-215-0db320afd912>:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  misogyny_results = misogyny_results.append({'Movie': movie_title, 'Cosine_Similarity': cos_similarity}, ignore_index=True)
<ipython-input-215-0db320afd912>:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  misogyny_results = misogyny_results.append({'Movie': movie_title, 'Cosine_Similarity': cos_similarity}, ignore_index=True)
<ipython-input-215-0db320afd912>:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  misogyny_results = misogyny_results.append({'Movie': movie_title, 'Cosine_Similarity': cos_similarity}, ignore_index=True)
<ipython-input-215-0db320afd912>:45: FutureWarning: The frame.append method is deprecated and will be removed from 

In [216]:
misogyny_results.head(20)

,Movie,Cosine_Similarity
0,Aan Milo Sajna,-1.000000
1,Chetna,-1.000000
2,Dharti,-1.000000
3,Geet,0.996546
4,Himmat,-1.000000
5,Humjoli,-1.000000
6,Ishq Par Zor Nahin,-1.000000
7,Jawab,-1.000000
8,Jeevan Mrityu,-1.000000
9,Kati Patang,0.878213


In [217]:
pdf_data['Movie'] = pdf_data['PDF_Name'].str.replace('.pdf', '')

<ipython-input-217-01ce1318170a>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  pdf_data['Movie'] = pdf_data['PDF_Name'].str.replace('.pdf', '')


In [218]:
result= misogyny_results[misogyny_results['Movie']=='Pink']
print(result)

Empty DataFrame
Columns: [Movie, Cosine_Similarity]
Index: []


# **IMAGE CLASSIFICATION**

**Creating an Image Classification Model**

In [220]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd

def load_images(directory):
    images = []
    filenames = []

    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path)
            img_array = np.array(img.resize((100, 100)))  # Resize images to a consistent size
            images.append(img_array.flatten())  # Flatten the image before appending
            filenames.append(filename)

    return np.array(images), filenames

# Load images and standardize the data
image_directory = '/content/drive/MyDrive/bollywood_data/Bollywood-Data-master/images-data/dir_002'
image_data, image_filenames = load_images(image_directory)

# Standardize the data
scaler = StandardScaler()
standardized_data = scaler.fit_transform(image_data)

# Apply KMeans clustering
num_clusters = 3  # Set the desired number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(standardized_data)

# Map cluster labels to classes
class_mapping = {0: 'Neutral', 1: 'nonmisogynistic', 2: 'misogynistic'}
df = pd.DataFrame({'Filename': image_filenames, 'Cluster': clusters})
df['Class'] = df['Cluster'].map(class_mapping)

# Display the DataFrme
print(df.head(10))







/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


                   Filename  Cluster            Class
0  Aan Milo Sajna_img_3.jpg        2     misogynistic
1         Aandhi_img_17.jpg        0          Neutral
2         Aandhi_img_14.jpg        1  nonmisogynistic
3         Aandhi_img_25.jpg        0          Neutral
4         Aandhi_img_23.jpg        0          Neutral
5         Aandhi_img_11.jpg        0          Neutral
6        Aakraman_img_1.jpg        2     misogynistic
7          Aandhi_img_2.jpg        1  nonmisogynistic
8        Aakraman_img_7.jpg        2     misogynistic
9         Aandhi_img_31.jpg        0          Neutral


**Extraction of Images from the Scripts pdf**

In [221]:
!pip install PyMuPDF

In [223]:
import os

def empty_directory(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f"Error deleting file {file_path}: {e}")

# Example usage to empty the output directory
output_directory = '/content/drive/MyDrive/bollywood_data/Bollywood-Data-master/images-data/scripts_extracted'
empty_directory(output_directory)

In [225]:
import fitz  # PyMuPDF
import os

def extract_images_from_pdf(pdf_path, output_directory):
    # Get the base name of the PDF file without extension
    pdf_basename = os.path.splitext(os.path.basename(pdf_path))[0]

    # Open the PDF file
    pdf_document = fitz.open(pdf_path)

    # Iterate through each page in the PDF
    for page_num in range(pdf_document.page_count):
        page = pdf_document[page_num]

        # Get the images on the page
        images = page.get_images(full=True)

        # Iterate through each image on the page
        for img_index, img_info in enumerate(images):
            image_index = img_info[0]
            base_image = pdf_document.extract_image(image_index)
            image_bytes = base_image["image"]

            # Save the image as a JPEG file
            image_filename = f"{pdf_basename}_page_{page_num + 1}_img_{img_index + 1}.jpg"
            image_path = os.path.join(output_directory, image_filename)

            with open(image_path, "wb") as image_file:
                image_file.write(image_bytes)

    # Close the PDF document
    pdf_document.close()

# Example usage for a directory containing PDF files
pdf_directory = '/content/drive/MyDrive/bollywood_data/Bollywood-Data-master/images-data/dir_002'
output_directory = '/content/drive/MyDrive/bollywood_data/Bollywood-Data-master/images-data/scripts_extracted'

# Iterate through each PDF file in the directory
for pdf_filename in os.listdir(pdf_directory):
    if pdf_filename.endswith(".pdf"):
        pdf_path = os.path.join(pdf_directory, pdf_filename)
        extract_images_from_pdf(pdf_path, output_directory)



In [226]:
from PIL import Image
import os
import numpy as np

def load_images(directory, target_size=(100, 100)):
    images = []
    filenames = []

    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            img_path = os.path.join(directory, filename)

            try:
                img = Image.open(img_path)
                img = img.convert("RGB")  # Ensure the image is in RGB mode
                img = img.resize(target_size)  # Resize images to a consistent size
                img_array = np.array(img)

                if img_array.shape == (*target_size, 3):  # Check if the shape is as expected
                    images.append(img_array.flatten())  # Flatten the image before appending
                    filenames.append(filename)
                else:
                    print(f"Ignoring image {filename} due to inconsistent shape: {img_array.shape}")
            except Exception as e:
                print(f"Ignoring image {filename} due to error: {e}")

    return np.array(images), filenames




**Classification of the extracted images**

In [229]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Load images and standardize the data
image_directory = '/content/drive/MyDrive/bollywood_data/Bollywood-Data-master/images-data/scripts_extracted'
image_data, image_filenames = load_images(image_directory)

# Check if there are enough samples for clustering
if len(image_data) < 3:
    print("Not enough samples for clustering.")
else:
    # Standardize the data
    scaler = StandardScaler()
    standardized_data = scaler.fit_transform(image_data)

    cluster = kmeans.fit_predict(standardized_data)

    # Map cluster labels to classes
    class_mapping = {0: 'Neutral', 1: 'Misogynistic', 2: 'Unmisogynistic'}
    predicted = pd.DataFrame({'Filename': image_filenames, 'Cluster': cluster})
    predicted['Class'] = predicted['Cluster'].map(class_mapping)

    # Display the DataFrame
    print(predicted.head(10))



Not enough samples for clustering.


**Assignment of class to Movies based on most frequent class labed on posters**

In [230]:
predicted.head()

,Filename,Cluster,Class,Movie
0,Haider_page_1_img_1.jpg,0,Neutral,Haider
1,Haider_page_14_img_1.jpg,2,Unmisogynistic,Haider
2,Haider_page_98_img_1.jpg,0,Neutral,Haider
3,Haider_page_122_img_1.jpg,1,Misogynistic,Haider
4,Haider_page_111_img_1.jpg,0,Neutral,Haider


In [231]:
# Assuming 'predicted' DataFrame contains the prediction results
class_mapping = {0: 'Neutral', 1: 'nonmisogynistic', 2: 'Misogynistic'}

# Extract movie names from 'Filename' and create 'Movie' column
predicted['Movie'] = predicted['Filename'].str.split('_', n=1, expand=True)[0]

# Group by movie and compute the mode of Cluster values
imagedf = predicted.groupby('Movie')['Cluster'].apply(lambda x: x.mode().iloc[0]).reset_index(name='Class')

# Map 'Class' values for the imagedf DataFrame
imagedf['Class'] = imagedf['Class'].map(class_mapping)

# Display the resulting DataFrame
print(imagedf)


                                         Movie            Class
0                                       Haider          Neutral
1                                      Maqbool          Neutral
2  Rang De Basanti Script - Film Companion-min  nonmisogynistic


**Usage of Adjectives and adverb dataset**

In [233]:
adjectives_df = pd.read_csv('/content/drive/MyDrive/bollywood_data/Bollywood-Data-master/wikipedia-data/female_adjectives.csv', sep='|', header=None, names=['Year', 'Adjectives'])



In [234]:
adjectives_df.head()

,Year,Adjectives
0,"1970,[young]",NaN
1,"1970,[]",NaN
2,"1970,[]",NaN
3,"1970,[]",NaN
4,"1970,[unrecognizable]",NaN


In [235]:
adjectives_df[['Year', 'Adjectives']] = adjectives_df['Year'].str.split(',', 1, expand=True)



<ipython-input-235-459968223531>:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  adjectives_df[['Year', 'Adjectives']] = adjectives_df['Year'].str.split(',', 1, expand=True)


In [236]:
adjectives_df.head()

,Year,Adjectives
0,1970,[young]
1,1970,[]
2,1970,[]
3,1970,[]
4,1970,[unrecognizable]


In [237]:
# Assuming 'your_column' is the column containing strings like [hi],[hello]
adjectives_df['Adjectives'] = adjectives_df['Adjectives'].str.replace(r'\[|\]', '').str.replace(',', ' ')


<ipython-input-237-1c1a003436ca>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  adjectives_df['Adjectives'] = adjectives_df['Adjectives'].str.replace(r'\[|\]', '').str.replace(',', ' ')


In [238]:
adjectives_df.head()

,Year,Adjectives
0,1970,young
1,1970,
2,1970,
3,1970,
4,1970,unrecognizable


In [239]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Download NLTK VADER lexicon
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

**Collection of positive and negative female adverbs and adjectives**

In [240]:
import nltk
nltk.download('vader_lexicon')

# Initialize the Sentiment Intensity Analyzer
sia = SentimentIntensityAnalyzer()

# Function to determine sentiment category
def get_sentiment_category(adjective):
    # Use VADER sentiment analysis
    sentiment_score = sia.polarity_scores(adjective)['compound']

    # Classify as Positive, Negative, or Neutral based on the compound score
    if sentiment_score >= 0.08:
        return 'Positive'
    elif sentiment_score <= -0.04:
        return 'Negative'
    else:
        return 'Neutral'

# Apply sentiment analysis and create a new column 'Sentiment_Category'
adjectives_df['Sentiment_Category'] = adjectives_df['Adjectives'].apply(get_sentiment_category)

# Separate texts for negative and positive adjectives
negative_text = ' '.join(adjectives_df[adjectives_df['Sentiment_Category'] == 'Negative']['Adjectives'])
positive_text = ' '.join(adjectives_df[adjectives_df['Sentiment_Category'] == 'Positive']['Adjectives'])

# Display the separated texts
print("Negative Adjectives:", negative_text)
print("Positive Adjectives:", positive_text)

Negative Adjectives: poor widowed blind disastrous violent widowed blind  young patriotic  unaware dumb young  poor teenage  widowed widowed sick drunk dominating poor devastated poor arrogant lost uptight unhappy ill shattered arrogant  prospective frantic grieving hapless sick widowed poor embarrassing frantic traditional  Indian  unaware insensitive widowed arrogant  spoiled poor poor widowed retarded arrogant reluctant arrogant  young guilty dumb challenged poor  orphan notorious frantic young  poor widowed unaware evil  strange old  tempered  short  aggressive lazy young  dead ill ill widowed tough dead rude poor poor evil snobby widowed poor sad poor injured  young dead insane lonely distressed  pregnant blind suicidal  heartbroken  pregnant greedy reluctant skeptical widowed insensitive docile  poor  twin  little frustrated anxious remorseless obsessive nagging broken biological  widowed frustrated  angry timid bored widowed drunk aggressive jealous depressed blind gullible viol

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [241]:
positive_text

'noble rich attractive  young young  innocent beautiful beautiful rich beautiful beautiful  young  married innocent rich  spoiled wealthy beautiful  young rich beautiful  young Handsome lovely  young rich wealthy beautiful  poor modern  delightful  soft good good wealthy lovely beautiful wealthy  gorgeous Kind rich wealthy beautiful  young rich playful lovely  pregnant attractive beautiful pretty rich beautiful  young intelligent pretty lively  young lovely beautiful  young beautiful  ugly beautiful  young wealthy young  dynamic modern  young  worthy  nameless sexy attractive rich pretty privileged lovely  attractive  young innocent rich beautiful intelligent ideal beautiful beautiful  young sweet beautiful caring devoted attractive  young beautiful  young  little lovely  beautiful  young pretty lovely  young beautiful wealthy beautiful young  rich beautiful powerful good successful beautiful beautiful rich  spoiled beautiful  young beautiful beautiful lovely wealthy innocent lovely  y

In [242]:
negative_text

'poor widowed blind disastrous violent widowed blind  young patriotic  unaware dumb young  poor teenage  widowed widowed sick drunk dominating poor devastated poor arrogant lost uptight unhappy ill shattered arrogant  prospective frantic grieving hapless sick widowed poor embarrassing frantic traditional  Indian  unaware insensitive widowed arrogant  spoiled poor poor widowed retarded arrogant reluctant arrogant  young guilty dumb challenged poor  orphan notorious frantic young  poor widowed unaware evil  strange old  tempered  short  aggressive lazy young  dead ill ill widowed tough dead rude poor poor evil snobby widowed poor sad poor injured  young dead insane lonely distressed  pregnant blind suicidal  heartbroken  pregnant greedy reluctant skeptical widowed insensitive docile  poor  twin  little frustrated anxious remorseless obsessive nagging broken biological  widowed frustrated  angry timid bored widowed drunk aggressive jealous depressed blind gullible violent dead dead drunk 

In [244]:
female_adverbs= pd.read_csv('/content/drive/MyDrive/bollywood_data/Bollywood-Data-master/wikipedia-data/female_adjectives.csv' , sep='|', header=None, names=['Year', 'Adverbs'])

In [245]:
female_adverbs.head()

,Year,Adverbs
0,"1970,[young]",NaN
1,"1970,[]",NaN
2,"1970,[]",NaN
3,"1970,[]",NaN
4,"1970,[unrecognizable]",NaN


In [246]:
female_adverbs[['Year', 'Adverbs']] = female_adverbs['Year'].str.split(',', 1, expand=True)

<ipython-input-246-197bc9ea5523>:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  female_adverbs[['Year', 'Adverbs']] = female_adverbs['Year'].str.split(',', 1, expand=True)


In [247]:
female_adverbs['Adverbs'] = female_adverbs['Adverbs'].str.replace(r'\[|\]', '').str.replace(',', ' ')

<ipython-input-247-0f705d5d83d6>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  female_adverbs['Adverbs'] = female_adverbs['Adverbs'].str.replace(r'\[|\]', '').str.replace(',', ' ')


In [248]:
female_adverbs.head()

,Year,Adverbs
0,1970,young
1,1970,
2,1970,
3,1970,
4,1970,unrecognizable


In [249]:
female_adverbs['Sentiment_Category'] = female_adverbs['Adverbs'].apply(get_sentiment_category)

# Separate texts for negative and positive adjectives
negative_text = ' '.join(female_adverbs[female_adverbs['Sentiment_Category'] == 'Negative']['Adverbs'])
positive_text = ' '.join(female_adverbs[female_adverbs['Sentiment_Category'] == 'Positive']['Adverbs'])

# Display the separated texts
print("Negative Adjectives:", negative_text)
print("Positive Adjectives:", positive_text)

Negative Adjectives: poor widowed blind disastrous violent widowed blind  young patriotic  unaware dumb young  poor teenage  widowed widowed sick drunk dominating poor devastated poor arrogant lost uptight unhappy ill shattered arrogant  prospective frantic grieving hapless sick widowed poor embarrassing frantic traditional  Indian  unaware insensitive widowed arrogant  spoiled poor poor widowed retarded arrogant reluctant arrogant  young guilty dumb challenged poor  orphan notorious frantic young  poor widowed unaware evil  strange old  tempered  short  aggressive lazy young  dead ill ill widowed tough dead rude poor poor evil snobby widowed poor sad poor injured  young dead insane lonely distressed  pregnant blind suicidal  heartbroken  pregnant greedy reluctant skeptical widowed insensitive docile  poor  twin  little frustrated anxious remorseless obsessive nagging broken biological  widowed frustrated  angry timid bored widowed drunk aggressive jealous depressed blind gullible viol

**Usage of plot dataset**

In [252]:
core_plot= pd.read_csv('/content/drive/MyDrive/bollywood_data/Bollywood-Data-master/wikipedia-data/coref_plot.csv')

In [253]:
core_plot.head()

,Unnamed: 0,Movie Name,Coref Plot
0,0,Aan Milo Sajna,Widowed and ailing Savitri Choudhury lives a wealthy lifestyle along with Savitri son Anil in a palatial mansion in India
1,1,Aan Milo Sajna,Savitri knows that Anil is only waiting for Savitri to die so that Anil can inherit the wealth and refuses to give Anil any money
2,2,Aan Milo Sajna,The Diwan convinces Anil to mend Anil ways get married and patch-up with Anil mom
3,3,Aan Milo Sajna,Shortly thereafter Anil does appear to have mended Anil ways gets involved in charity and even introduces a young woman named Deepali to Anil mom
4,4,Aan Milo Sajna,Deepali moves in the mansion looks after Savitri so much so that Savitri decides to make Savitri the sole beneficiary of Savitri estate


In [254]:
core_plot.rename(columns={'Movie Name': 'Movie'}, inplace=True)
core_plot.rename(columns={'Coref Plot': 'Coref'}, inplace=True)

In [255]:
core_plot.columns

Index(['Unnamed: 0', 'Movie', 'Coref'], dtype='object')

In [256]:
# Group by lowercase movie names and concatenate the 'Coref' values
core_plot = core_plot.groupby(core_plot['Movie'].str.lower())['Coref'].apply(lambda x: ' '.join(x)).reset_index()

# Display the resulting DataFrame




In [257]:
core_plot.head(10)

,Movie,Coref
0,100 days,The film opens with a young woman Devi who gets sudden vision of incidents and accidents that are yet to happen Devi has a vision of Devi sister Rama being murdered Devi college friends Sudha Mathur and Sunil try to help Sudha sort through Sudha visions but to little avail Devi is relieved after Devi talks to Devi sister and finds that Devi is alive However just some time later Rama is murdered in the same way as Devi had pictured The murderer hides Rama 's body in a mansion 's wall Rama is reported missing Devi firmly believes Rama is dead Five years later Devi moves to Devi uncle 's home where Devi eventually meets and is courted by millionaire businessman Ram Kumar Sunil who was secretly in love with Devi is deeply disappointed Devi and Ram marry and enter Devi family mansion which Devi has re-earned after a legal battle Little known to anybody this is the same mansion where Rama was buried When Devi starts having the visions again Rama 's skeleton is not the only thing that will come tumbling out Devi sees a wall in the mansion being torn down Devi tears down the wall only to find a skeleton tumbling out of it Devi knows whose skeleton it is : it 's Rama 's since Devi had a necklace similar to Devi 's Also the dead woman 's skeleton is roughly same height as that of Rama The Inspector quickly points out that since the mansion was closed when Rama disappeared anyone could have hidden a dead body in there and nobody would know the truth However Rama doubts that the dead woman is Rama : Several such necklaces are available Devi now gets another vision of another woman getting murdered Devi also pinpoints two details : a magazine named Priya with a horse on Priya cover page and a video cassette labelled 100 Days Sunil and Devi visit the weekly magazine office The editor politely informs that the next six months ' covers do not feature any equestrian theme whatsoever The video cassette clue too is a dead end : no video stor...
1,13b,Manohar moves into a new apartment 13B on the 13th floor with Manohar family fulfilling Manohar life 's biggest dream But encounter a series of small but strange incidents which are regarded as inauspicious by Manohar family but shrugged off by an excited Manohar The elevator in the apartment works for everyone in the building except Manohar which bothers Manohar The women in the family get hooked onto a new TV show Yavarum Nalam The show is about a family eerily similar to Manohar 's who have also just moved into a new house like they have As the serial unfolds Manohar notices that the incidents that happen in the serial are a reflection of what is happening to his family Things like his sister graduating his wife getting pregnant and later suffering a miscarriage are all also shown in the serial Priya is saved by Priya longtime family friend and doctor Dr Balu The rest of the family remains oblivious to the similarities and Manohar prefers that way to avoid panic also notices that Manohar camera takes distorted pictures of while is in the apartment but not while is outside the apartment Things start taking a turn for the worse in the serial and Manohar becomes terrified that the same might happen to family Manohar now wants to know who is behind the making of the TV show then unravels a terrifying secret : the apartment live in shared address with a house where a family of eight was butchered back in 1977 was the family of a TV news anchor named Chitra On the day of Chitra 's engagement an ardent fan of Chitra wanted to stop the engagement and failed in doing so Disheartened committed suicide Subsequently all members of Chitra 's family were murdered with a hammer in a single day The blame rested on the lunatic brother in the family as had been seen with the hammer last The police officer investigating the case also committed suicide by hanging in the same house Manohar and Manohar policeman friend Shiva meet the madman brother ...
2,16 december,A four member team -- Vir Vi

In [258]:
script= core_plot[core_plot['Movie']=='rockstar']

In [259]:
script

,Movie,Coref
1477,rockstar,A large crowd gathers outside the Verona Arena in Italy with eager fans queuing to watch the performance of Jordan a rock musician Despite being mobbed Jordan manages to escape and reach the arena Jordan shows a complete disregard for Jordan fans storms onto the stage in a fit of rage and starts strumming angrily on Jordan guitar A flashback reveals the story of Jordan whose real name is Janardhan Jhakar a k a JJ a simple Jat boy from Delhi JJ is a student at Hindu College who dreams of becoming an international star and this dream is often laughed at by friends and class mates Khatana the canteen owner tells JJ that the only way a meaningful melody can be produced is by undergoing trauma via a heartbreak JJ jokingly tells that since has n't yet seen bad times wo n't be able to reach idol 's level of fame goes to the performance of Heer Kaul a beautiful Kashmiri Pandit girl from St Stephens College and overhears that she is the `` perfect heartbreaking machine Instantly deciding to propose to JJ suffers humiliation which JJ deciphers as the perfect method for receiving the heart-break needed to achieve stardom However over time become good friends often going on fun adventures together and sometimes engaging in behavior which the conservative JJ thinks is risky Heer becomes engaged to Jai and ultimately marries in Kashmir after which the couple depart to Prague During this time develops feelings for JJ and coins the nickname `` Jordan for interviews before leaving JJ returns home finds thrown out by elder brother for attending Heer 's wedding and allegedly stealing 500000 - something which did not do takes refuge for two months in the nearby Nizamuddin Dargah singing qawwalis and then moves into Khatana 's home Ustad Jameel Khan a Padma Bhushan classical artist remembers JJ singing at the Dargah and prompts Dhingra the owner of `` Platinum Records to sign JJ as a future star Sheena a journalist interviews JJ and hin...


In [260]:
rockstar_coref = core_plot[core_plot['Movie'] == 'rockstar']['Coref'].values

In [261]:
rockstar_combined_coref = ' '.join(rockstar_coref)
rockstar_combined_coref

" A large crowd gathers outside the Verona Arena in Italy  with eager fans queuing to watch the performance of Jordan  a rock musician  Despite being mobbed  Jordan manages to escape and reach the arena  Jordan shows a complete disregard for Jordan fans  storms onto the stage in a fit of rage and starts strumming angrily on Jordan guitar  A flashback reveals the story of Jordan  whose real name is Janardhan Jhakar a k a  JJ  a simple Jat boy from  Delhi  JJ is a student at Hindu College who dreams of becoming an international star  and this dream is often laughed at by friends and class mates  Khatana  the canteen owner  tells JJ that the only way a meaningful melody can be produced is by undergoing trauma via a heartbreak  JJ jokingly tells that since has n't yet seen bad times  wo n't be able to reach idol 's level of fame  goes to the performance of Heer Kaul  a beautiful Kashmiri Pandit girl from St Stephens College  and overhears that she is the `` perfect heartbreaking machine   

**Extraction of adverbs and adjectives from scripts pdf**

In [262]:
pdf_data

,PDF_Name,Extracted_Text,new,vectors,Movie
0,haider.pdf,FADE I I is he mids and he insurgency in is a is peak he defence is i o he which are by and by o his challenge he formed he a of and he play a key role in reducing in he valley he decided o hold he six o show he world ha are under in FADE I DAY On a lonely road wo of and a couple of of AK s peep he of a barbed bunker an building in a posh area of SUPER BUREAU ROAD A conference room BRIGADIER a handful of senior army and police BRIGADIER Bulbul insurgency force Bulbul We have o arm fund and run he force able is full of BRIGADIER k individual k f he will be he are our Any individual or group anyone he proxy war by he and be hey or will be our primary he will be senior cop he General Police o pronounce i b a and BRIGADIER he army will run he bu your will be crucial in and My are you Brigadier We see urn o He a smug smile BRIGADIER he Bulbul sar singing Is and kill O EX CHAMBER DAY A man dressed in a and wearing an ha of his window his is a handsome lawyer in his An old man behind him for him o speak k Was he in or MA d d us j us was he a One gave him a weapon o hide he same guy and informed he army of he and a him Armed Special God in he heaven and he army he And he army is he Armed Special AcOLD MA j us my son is been a he cinema How can I help MA Please ge him d Uncle I have no o free your boy I can ge him bu I can possibly save his life We have o move him of he camp and ge him o a prison Well need o a formal We need o ge him a a deep Ill be a false would ge him a hearing in a Once he is in he can disappear He he man closer k So wha shall i be Public or Arms Possession he old man is confused O EX OF A Gypsy by wo is moving on he of d d wo one is arms possession and he over an envelope o and he aside He a bag from he side I have wo for you inside he bag and a bole of Vodka His smile He inside he bag o look for he second hing he second one Chance of a a him O EX HOUSE DAY he Gypsy comes o a in of an abandoned house in he he house is surrounded by ask force he house...,fade mids insurgency peak defence challenge formed play key role reducing valley decided hold six show world ha fade day lonely road wo couple ak peep barbed bunker building posh area super bureau road conference room brigadier handful senior army police brigadier bulbul insurgency force bulbul arm fund run force able full brigadier k individual k f individual group anyone proxy war hey primary senior cop general police pronounce b brigadier army run bu crucial brigadier see urn smug smile brigadier bulbul sar singing kill ex chamber day man dressed wearing ha window handsome lawyer old man behind speak k wa u j u wa one gave weapon hide guy informed army armed special god heaven army army armed special acold j u son cinema help please ge uncle free boy ge bu possibly save life move camp ge prison well need formal need ge deep ill false would ge hearing disappear man closer k wha shall public arm possession old man confused ex gypsy wo moving wo one arm possession envelope aside bag side wo inside bag bole vodka smile inside bag look second hing second one chance ex house day gypsy come abandoned house house surrounded ask force house way b koku b weapon paisa power need nourish fund come house officer sir early morning sir early morning asleep single sho red group officer war sir house young corner heir fee kalashnikov belonging hem many gun f bu would hey ake arm heir wall making door glass come sep back sudden shock ring man prison exchange kalashnikov back commando sanding wha suddenly killing k dead police sar dress crime scene rope dead heir call medium b meaning back ie day old house somewhere heavily area known voice man screaming pain bearded arm screaming pain holding wo men anxiously oh young men slung heir also aside severe yn yn case severe need operae look back ge face risk going b k ake risk ha commander explosion hey hold look deep arrange ambulance lake bag leave bu ex dal lake back wo young me

In [263]:
import spacy
spacy.cli.download("en_core_web_sm")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [264]:
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_sm")
def extract_adj_adv(text):
    doc = nlp(text)
    adjectives = [token.text for token in doc if token.pos_ == "ADJ"]
    adverbs = [token.text for token in doc if token.pos_ == "ADV"]
    return adjectives, adverbs

# Apply the function to each row
pdf_data[['Movie_adj', 'Movie_adv']] = pdf_data['new'].apply(lambda x: pd.Series(extract_adj_adv(x)))

In [265]:
pdf_data.head()

,PDF_Name,Extracted_Text,new,vectors,Movie,Movie_adj,Movie_adv
0,haider.pdf,FADE I I is he mids and he insurgency in is a is peak he defence is i o he which are by and by o his challenge he formed he a of and he play a key role in reducing in he valley he decided o hold he six o show he world ha are under in FADE I DAY On a lonely road wo of and a couple of of AK s peep he of a barbed bunker an building in a posh area of SUPER BUREAU ROAD A conference room BRIGADIER a handful of senior army and police BRIGADIER Bulbul insurgency force Bulbul We have o arm fund and run he force able is full of BRIGADIER k individual k f he will be he are our Any individual or group anyone he proxy war by he and be hey or will be our primary he will be senior cop he General Police o pronounce i b a and BRIGADIER he army will run he bu your will be crucial in and My are you Brigadier We see urn o He a smug smile BRIGADIER he Bulbul sar singing Is and kill O EX CHAMBER DAY A man dressed in a and wearing an ha of his window his is a handsome lawyer in his An old man behind him for him o speak k Was he in or MA d d us j us was he a One gave him a weapon o hide he same guy and informed he army of he and a him Armed Special God in he heaven and he army he And he army is he Armed Special AcOLD MA j us my son is been a he cinema How can I help MA Please ge him d Uncle I have no o free your boy I can ge him bu I can possibly save his life We have o move him of he camp and ge him o a prison Well need o a formal We need o ge him a a deep Ill be a false would ge him a hearing in a Once he is in he can disappear He he man closer k So wha shall i be Public or Arms Possession he old man is confused O EX OF A Gypsy by wo is moving on he of d d wo one is arms possession and he over an envelope o and he aside He a bag from he side I have wo for you inside he bag and a bole of Vodka His smile He inside he bag o look for he second hing he second one Chance of a a him O EX HOUSE DAY he Gypsy comes o a in of an abandoned house in he he house is surrounded by ask force he house...,fade mids insurgency peak defence challenge formed play key role reducing valley decided hold six show world ha fade day lonely road wo couple ak peep barbed bunker building posh area super bureau road conference room brigadier handful senior army police brigadier bulbul insurgency force bulbul arm fund run force able full brigadier k individual k f individual group anyone proxy war hey primary senior cop general police pronounce b brigadier army run bu crucial brigadier see urn smug smile brigadier bulbul sar singing kill ex chamber day man dressed wearing ha window handsome lawyer old man behind speak k wa u j u wa one gave weapon hide guy informed army armed special god heaven army army armed special acold j u son cinema help please ge uncle free boy ge bu possibly save life move camp ge prison well need formal need ge deep ill false would ge hearing disappear man closer k wha shall public arm possession old man confused ex gypsy wo moving wo one arm possession envelope aside bag side wo inside bag bole vodka smile inside bag look second hing second one chance ex house day gypsy come abandoned house house surrounded ask force house way b koku b weapon paisa power need nourish fund come house officer sir early morning sir early morning asleep single sho red group officer war sir house young corner heir fee kalashnikov belonging hem many gun f bu would hey ake arm heir wall making door glass come sep back sudden shock ring man prison exchange kalashnikov back commando sanding wha suddenly killing k dead police sar dress crime scene rope dead heir call medium b meaning back ie day old house somewhere heavily area known voice man screaming pain bearded arm screaming pain holding wo men anxiously oh young men slung heir also aside severe yn yn case severe need operae look back ge face risk going b k ake risk ha commander explosion hey hold look deep arrange ambulance lake bag leave bu ex dal l

# **LLM**

**Using LLMs to improve movie plots reducing gender bias and misogyny remarks**

In [266]:
!pip install langchain
!pip install langchain_core
!pip install langchain_openai

In [267]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("you're a movie critic and can analyze movie plots to identify gender bias and mysognistic remarks. Given a movie ")
model = ChatOpenAI()
chain = prompt | model

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

# for given script,
1.   adverb adj
2.   labelled class
3.   plot if available
4.   image class if available





In [269]:
#we have postive and negative adj and adv for female class

In [270]:
#our model will take 2 or more inputs to produce an output that gives a better script idea for the future

In [271]:
#this is done by substituting the negative ad verbs and adjectives using positive ones, making the plot gender neutral/suggestions, possible future booming ideas etc.

In [272]:
positive_text #list of positive adj and adverbs

'noble rich attractive  young young  innocent beautiful beautiful rich beautiful beautiful  young  married innocent rich  spoiled wealthy beautiful  young rich beautiful  young Handsome lovely  young rich wealthy beautiful  poor modern  delightful  soft good good wealthy lovely beautiful wealthy  gorgeous Kind rich wealthy beautiful  young rich playful lovely  pregnant attractive beautiful pretty rich beautiful  young intelligent pretty lively  young lovely beautiful  young beautiful  ugly beautiful  young wealthy young  dynamic modern  young  worthy  nameless sexy attractive rich pretty privileged lovely  attractive  young innocent rich beautiful intelligent ideal beautiful beautiful  young sweet beautiful caring devoted attractive  young beautiful  young  little lovely  beautiful  young pretty lovely  young beautiful wealthy beautiful young  rich beautiful powerful good successful beautiful beautiful rich  spoiled beautiful  young beautiful beautiful lovely wealthy innocent lovely  y

In [273]:
negative_text #list of negative adj and adverbs

'poor widowed blind disastrous violent widowed blind  young patriotic  unaware dumb young  poor teenage  widowed widowed sick drunk dominating poor devastated poor arrogant lost uptight unhappy ill shattered arrogant  prospective frantic grieving hapless sick widowed poor embarrassing frantic traditional  Indian  unaware insensitive widowed arrogant  spoiled poor poor widowed retarded arrogant reluctant arrogant  young guilty dumb challenged poor  orphan notorious frantic young  poor widowed unaware evil  strange old  tempered  short  aggressive lazy young  dead ill ill widowed tough dead rude poor poor evil snobby widowed poor sad poor injured  young dead insane lonely distressed  pregnant blind suicidal  heartbroken  pregnant greedy reluctant skeptical widowed insensitive docile  poor  twin  little frustrated anxious remorseless obsessive nagging broken biological  widowed frustrated  angry timid bored widowed drunk aggressive jealous depressed blind gullible violent dead dead drunk 

In [274]:
imagedf #has movie posters with their class

,Movie,Class
0,Haider,Neutral
1,Maqbool,Neutral
2,Rang De Basanti Script - Film Companion-min,nonmisogynistic


In [276]:
llm_data = pdf_data[['Movie', 'Movie_adj', 'Movie_adv']]


In [277]:
llm_data['movie_attr']= llm_data['Movie_adj'] + llm_data['Movie_adv']
columns_to_drop= ['Movie_adj','Movie_adv']

<ipython-input-277-3e07017c07b7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  llm_data['movie_attr']= llm_data['Movie_adj'] + llm_data['Movie_adv']


In [278]:
llm_data= llm_data.drop(columns= columns_to_drop)

In [279]:
llm_data

,Movie,movie_attr
0,haider,"[key, lonely, barbed, posh, senior, able, full, individual, primary, senior, general, crucial, smug, handsome, old, free, formal, ill, false, closer, public, old, inside, second, second, early, early, asleep, single, many, sudden, dead, dead, ie, old, bearded, young, slung, severe, severe, deep, young, inner, young, main, surgical, silen, surgical, close, lower, able, able, small, nigh, young, large, full, bloody, daw, blue, closer, long, armed, several, mosque, sleepy, sound, daw, inner, worse, back, careful, young, daw, armed, close, armed, black, hooded, major, young, young, second, young, back, young, major, back, major, silen, ready, ense, wooden, major, dead, corridor, corridor, major, dead, angry, ...]"
1,highway,"[plain, rich, bungalow, small, much, good, ye, safe, hum, funny, main, inside, fresh, second, main, second, hard, jeep, rear, inside, unable, main, unbearable, inside, brisk, chal, main, brough, fainly, jeep, rear, jeep, free, massive, harder, pu, dead, closer, senseless, small, ominous, full, final, main, main, goli, main, hum, high, chal, chal, chal, chal, sudden, old, old, old, jeep, jeep, jeep, back, rap, rear, loud, rocky, many, rear, long, bu, various, crowded, seen, dark, rear, hum, hum, hum, confused, nervousness, open, rear, quick, large, side, high, narrow, sound, open, back, dark, second, big, big, dead, moral, back, small, compound, small, confused, ...]"
2,jabwemet,"[blank, mid, long, sound, able, aged, able, loud, hum, black, open, key, many, ense, snap, key, bride, key, blank, diffused, ie, beggar, final, inaudible, loud, full, young, sill, young, loose, second, ye, young, back, small, young, main, main, young, main, mere, main, second, main, lower, main, crowded, hosel, ye, main, siren, hard, young, nigh, gee, mere, main, closer, main, main, main, main, main, main, usual, small, unscheduled, back, full, open, omao, ready, se, main, sill, bloody, loud, main, main, mere, mere, main, main, small, rear, many, low, melodious, mad, mad, old, nervous, ye, main, sleepy, jumble, maximum, main, small, free, ...]"
3,kaminey,"[black, sound, sleepy, small, local, local, old, able, able, young, senior, old, small, senior, magical, jeep, full, wiry, underway, main, mock, local, playful, open, scuffle, mock, local, dark, dark, heavy, local, inside, crumple, main, square, young, close, dead, sudden, brief, several, closer, dark, fellow, rickle, closer, dead, burn, close, young, young, ruffian, parallel, visual, amazing, visual, sharp, mere, fashionable, red, various, playful, mere, final, large, red, heeled, reddy, square, large, red, new, main, close, blue, impassive, hum, double, hard, jockey, cold, emerge, new, able, finer, small, round, double, mere, hum, om, small, black, mere, chal, young, back, loud, far, loud, ...]"
4,nil battey sannata,"[yellow, early, early, early, early, open, old, small, wirl, asleep, closed, asleep, early, pu, early, sleepy, open, early, small, new, black, early, various, early, regular, ea, regular, early, ready, black, facial, open, open, wide, fake, able, mischievous, main, able, ouch, par, new, double, full, main, hum, sound, principal, principal, happy, corridor, narrow, happy, able, physic, wide, serial, ready, early, sill, uniform, ready, able, able, blue, main, blue, able, serious, angry, bole, main, asleep, main, small, ill, huge, nervous, gingerly, deep, older, angry, regular, main, main, blue, small, open, wide, red, main, deep, able, serious, main, serious, main, main, main, early, ...]"
5,queen,"[final, slow, many, many, happy, main, rani, ammo, handsome, many, old, young, old, main, many, yellow, mere, main, main, dari, nice, grey, main, fancy, unable, main, sab, main, main, modern, sab, main, bolo, concerned, sure, sill, various, rani, worried, rickshaw, rani, rear, new, main, simpler, bad, miny, close, new, main, rani, simple, full, sill, oes, much, sick, sole, good, bad, married, main, oblivious, single, main, full, busy, mobile, pu, abl

In [280]:
#llm data has the movie name the adverbs and adjects in the column attributes

In [281]:
#script df has the movie name and the coref plot

In [282]:
rockstar_combined_coref #has coref plot as a text

" A large crowd gathers outside the Verona Arena in Italy  with eager fans queuing to watch the performance of Jordan  a rock musician  Despite being mobbed  Jordan manages to escape and reach the arena  Jordan shows a complete disregard for Jordan fans  storms onto the stage in a fit of rage and starts strumming angrily on Jordan guitar  A flashback reveals the story of Jordan  whose real name is Janardhan Jhakar a k a  JJ  a simple Jat boy from  Delhi  JJ is a student at Hindu College who dreams of becoming an international star  and this dream is often laughed at by friends and class mates  Khatana  the canteen owner  tells JJ that the only way a meaningful melody can be produced is by undergoing trauma via a heartbreak  JJ jokingly tells that since has n't yet seen bad times  wo n't be able to reach idol 's level of fame  goes to the performance of Heer Kaul  a beautiful Kashmiri Pandit girl from St Stephens College  and overhears that she is the `` perfect heartbreaking machine   